In [ ]:
import os
import numpy
from matplotlib import pyplot

In [ ]:
%matplotlib inline

In [ ]:
%run analysis.py

In [ ]:
freq_channel = 300
time_integration = 0
bad_ants=[0, 2, 11, 24, 50, 53, 54, 67, 69, 98,122, 136, 139]

In [ ]:
mdm_fn = "/Users/matyasmolnar/Downloads/HERA_Data/robust_cal/zen.2458098.43869.HH.uvh5"
if os.path.exists(mdm_fn):
    filename = mdm_fn
else:
    filename = "./zen.2458098.43869.HH.uvh5"

In [ ]:
hdraw = HERAData(filename)
reds = get_reds(hdraw.antpos, pols=['ee'])
data, flags, nsamples = hdraw.read(freq_chans=[freq_channel])

In [ ]:
flt_bls = fltBad(reds, bad_ants)
cRedG=groupBls(flt_bls)
# This groups reundant baselines. Format is [ [groupid, i, j]]
print(cRedG[:5])

In [ ]:
# Collect the data together
cData=numpy.hstack([data[(zz[1], zz[2], "ee")] for zz in cRedG])

In [ ]:
print(cData.shape)
fig, ax = pyplot.subplots(figsize=(12,4))
ax.matshow(cData.real, aspect='auto')

In [ ]:
# Finding all the antennas used in our flagged data
ants = numpy.unique(cRedG[:,1:])
no_unq_bls = len(numpy.unique(cRedG[:, 0]))

In [ ]:
# Setup initial condition
xvis=numpy.ones(no_unq_bls*2) # Number of unique baselines; complex vis
xgains=numpy.ones(len(ants)*2) # Complex gain

In [ ]:
# This creates a compiled version of likelihood where the first
# parameter (the redundant groups) and second (the error distribution)
# has allready be filled in [-- this is "partial" fn application]
ff=jit(functools.partial(vgLkl, numpy.array(relabelAnts(cRedG)), "cauchy"))
res1=scipy.optimize.minimize(ff, numpy.hstack([xvis, xgains]), args=(cData[time_integration, :],), jac=jacrev(ff))

In [ ]:
# Minimizing without the JAX automatic jacobian
res2=scipy.optimize.minimize(ff, numpy.hstack([xvis, xgains]), args=(cData[time_integration, :],))

In [ ]:
def split_results(results):
    ''' Split results from minimization into visibility and gains arrays '''
    vis_params, gains_params = numpy.split(results['x'], [no_unq_bls*2,])
    vis_params = vis_params.reshape((-1, 2))
    gains_params = gains_params.reshape((-1, 2))
    res_vis = vis_params[:,0] + 1j*vis_params[:,1]
    res_gains = gains_params[:,0] + 1j*gains_params[:,1]
    return res_vis, res_gains

In [ ]:
res1_vis, res1_gains = split_results(res1)

In [ ]:
res2_vis, res2_gains = split_results(res2)

In [ ]:
np.absolute(res1_vis - res2_vis)/np.sqrt(np.absolute(res1_vis*res2_vis))

In [ ]:
np.absolute(res1_gains - res2_gains)/np.sqrt(np.absolute(res1_gains*res2_gains))